# Modules

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

import numpy as np

import matplotlib.pyplot as plt

import warnings
from datetime import datetime

import pandas as pd



from ta.momentum import stochrsi,rsi
from ta.trend import ema_indicator, macd_diff, vortex_indicator_neg, vortex_indicator_pos, adx, cci, sma_indicator
from ta.volatility import bollinger_hband, bollinger_lband
from ta.volume import volume_weighted_average_price


warnings.filterwarnings('ignore')

# Utils

In [ ]:

def addIndicators(df:pd.DataFrame,b_engulfings:bool=False, derivative:bool=False, double_derivative:bool=False) -> pd.DataFrame:
    """Apply indicators to the DataFrame.

    Args:
        df (pd.DataFrame): The dataframe you want to add indicators on.
        b_engulfings (bool, optional): Add bearish and bullish engulfing indicators. Defaults to False.
        derivative (bool, optional): Add the first derivative of the Close price. Defaults to False.
        double_derivative (bool, optional): Add the second derivative of the Close price. Defaults to False.

    Returns:
        pd.DataFrame: The same dataframe with indicators
    """
    def isBearishCandleStick(candle) -> bool:
        """Check whether a candle is a bearish candle or not

        Args:
            candle (pd.Series): The current candle that contains OHLC

        Returns:
            bool: A boolean representing if the candle is bearish candle (True) or not (False)
        """
        return candle['Close']<candle['Open']

    def isBullishCandleStick(candle) -> bool:
        """Check whether a candle is a bullish candle or not

        Args:
            candle (pd.Series): The current candle that contains OHLC

        Returns:
            bool: A boolean representing if the candle is bullish candle (True) or not (False)
        """
        return candle['Close']>candle['Open']

    def isBullishEngulfing(previous_candle,current_candle) -> int:
        """A function that check for bullish engulfing pattern through candle stick

        Args:
            previous_candle (pd.Series): The previous candle that contains OHLC
            current_candle (pd.Series): The current candle that contains OHLC

        Returns:
            int: represent the pattern spotting : 1 bullish engulfing, 0 not.
        """
        return 1 if isBearishCandleStick(previous_candle) and isBullishCandleStick(current_candle) and previous_candle['Open']<current_candle['Close'] and previous_candle['Close']>current_candle['Open'] else 0
        
    def isBearishEngulfing(previous_candle,current_candle) -> int:
        """A function that check for bearish engulfing pattern through candle stick

        Args:
            previous_candle (pd.Series): The previous candle that contains OHLC
            current_candle (pd.Series): The current candle that contains OHLC

        Returns:
            int: represent the pattern spotting : 1 bearish engulfing, 0 not.
        """
        return 1 if isBullishCandleStick(previous_candle) and isBearishCandleStick(current_candle) and previous_candle['Close']<current_candle['Open'] and previous_candle['Open']>current_candle['Close'] else 0    
            
    df['High_Low_diff'] = df.High-df.Low
    df['EMA20'] = ema_indicator(df.Close,20)
    df['EMA50'] = ema_indicator(df.Close,50)
    df['EMA100'] = ema_indicator(df.Close,100)
    df['EMA200'] = ema_indicator(df.Close,200)
    df['MACD'] = macd_diff(df.Close)
    df['Stoch_RSI'] = stochrsi(df.Close, 14, smooth1=3, smooth2=3)
    df['Vortex'] = (vortex_indicator_pos(df.High,df.Low,df.Close,20,fillna=True)-1)-(vortex_indicator_neg(df.High,df.Low,df.Close,20,fillna=True)-1)
    df['Bollinger_low'] = bollinger_hband(df.Close,20,fillna=True)
    df['Bollinger_high'] = bollinger_lband(df.Close,20,fillna=True)
    df['ADX'] = adx(df.High,df.Low,df.Close)
    df['CCI'] = cci(df.High,df.Low,df.Close,14)
    df['OVB'] = (np.sign(df.Close.diff())*df.Volume).fillna(0).cumsum()
    df['OVB_EMA200'] = ema_indicator(df.OVB,200)
    trixLength = 9
    trixSignal = 21
    df['TRIX'] = ema_indicator(ema_indicator(ema_indicator(df['Close'], window=trixLength), window=trixLength), window=trixLength)
    df['TRIX_PCT'] = df["TRIX"].pct_change()*100
    df['TRIX_SIGNAL'] = sma_indicator(df['TRIX_PCT'],trixSignal)
    df['TRIX_HISTO'] = df['TRIX_PCT'] - df['TRIX_SIGNAL']
    #df['EVM'] = ease_of_movement(df.High,df.Low,df.Volume,14)
    if b_engulfings==True:
        df['Bullish_engulfing'] = np.nan
        df['Bullish_engulfing'].iloc[1:] = [isBullishEngulfing(df.iloc[i-1],df.iloc[i]) for i in range(1,len(df))]
        df['Bearish_engulfing'] = np.nan
        df['Bearish_engulfing'].iloc[1:] = [isBearishEngulfing(df.iloc[i-1],df.iloc[i]) for i in range(1,len(df))]
    if derivative==True:
        df['Slope'] = df.Close.diff()
    if double_derivative==True:
        df['Acceleration'] = df.Close.diff().diff()
    return df.dropna()

In [ ]:
df = pd.read_csv('../backtest_tools/database/database/Binance/15m/BTC-USDT.csv',names=['Date','Open','High','Low','Close','Volume'])
df = df.iloc[1:]
df['Date'] = df['Date'].apply(lambda x: int(str(x)[:-3]))
df = df.astype(float)
df['Timestamp'] = df['Date'].astype(int)
df['Date'] = df['Date'].astype(int).apply(datetime.fromtimestamp)
df = df.set_index('Date')
df = addIndicators(df)
df.head()

In [ ]:
features = df[['Open', 'High', 'Low', 'Close', 'Volume', 'High_Low_diff',
       'EMA20', 'EMA50', 'EMA100', 'EMA200', 'MACD', 'Stoch_RSI', 'Vortex',
       'Bollinger_low', 'Bollinger_high', 'ADX', 'CCI', 'OVB', 'OVB_EMA200',
       'TRIX', 'TRIX_PCT', 'TRIX_SIGNAL', 'TRIX_HISTO']].iloc[:-1]

labels = ((df['Close']-df['Close'].mean())/df['Close'].std()).iloc[1:]

X_train, X_test, y_train, y_test = train_test_split(features,labels,test_size=0.5,shuffle=True)

model = RandomForestRegressor(n_estimators=120,criterion='absolute_error',).fit(X_train,y_train)
mean_squared_error(model.predict(X_test),y_test,squared=False)


In [ ]:
df['Pred_close'] = np.nan
df['Pred_close'].iloc[1:]=(model.predict(features)*df['Close'].std())+df['Close'].mean()
fig, ax_left = plt.subplots(1, figsize=(25,8))

ax_left.plot(df.Close[-200:], color='red',label='Close')
ax_left.plot(df.Pred_close[-200:], color='blue',label='Close pred')
ax_left.legend(loc="upper left")#